In [10]:
from pyspark.sql.functions import date_format, countDistinct

import utils as u
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("entropy").config("spark.sql.session.timeZone", "+11").getOrCreate()


In [29]:
from scipy.stats import entropy
monthly_trans = u.read_tables(spark, "transactions")
monthly_trans = monthly_trans.withColumn("order_month", 
                                    date_format('order_datetime','yyyy-MM'))
monthly = monthly_trans.groupBy("merchant_abn", "order_month").count()

monthly = monthly.toPandas()
a = monthly["merchant_abn"].unique().tolist()

entropies = {}
for abn in a:
    
    this_merchant = monthly.loc[monthly['merchant_abn'] == abn]
    by_month = this_merchant["count"]
    entropies[abn] = entropy(by_month)

computed = spark.createDataFrame(entropies, schema=["merchant_abn", "month_entrophy"])
computed.limit(5)

TypeError: Can not infer schema for type: <class 'int'>

4422